# Model Evaluation of a Naive Bayes Classification Model
Noah MacGillivray | BIT-446 Assignment(T5) | Summer 2025  
**Environment:** Ubuntu, VS Code

## Imports 

In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import os 
import matplotlib as plt 
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Data Import

In [ ]:
failure_rate = pd.read_excel('your_file.xlsx', nrows=100)  # read first 100 rows
